In [1]:
#imports
import random as rd
import math
import numpy as np
import sys
import copy

#imports for experimentation and file handling
import IOHexperimenter as IOH
from IOHexperimenter import IOH_function
from zipfile import ZipFile
import os
from os.path import basename
import shutil
from shutil import make_archive
import datetime

In [2]:
#class for particle objects
class Particle:
    def __init__(self, dim):
        
        self.d = dim #number of variables
        
        # position, velocity and fitness as instance variables
        
        self.position = np.zeros(self.d) # particle position
        self.velocity = np.zeros(self.d) # particle velocity
        self.fitness = np.inf            # fitness individual
        self.pos_best = np.zeros(self.d) # best position individual
        self.best_fitness = np.inf       # best fitness individual

        # initialize particle with uniform random position and velocities
        for i in range (0,self.d):
            self.velocity[i] = rd.uniform(-1,1)
            self.position[i] = rd.uniform(-5, 5)
            
    # update particle velocity
    def update_velocity(self,pos_best_g, w):
        
        c1 = 1.4944       # cognative constant
        c2 = 1.4944       # social constant

        # create uniform random vectors between 0 and c1 / c2
        u1 = np.random.uniform(0,c1, self.d)
        u2 = np.random.uniform(0,c2, self.d)
        
        """ update velocity with previous velocity and component-wise multiplication of u-vectors 
        with differences between current solution, previous best and global best"""
        for i in range (0,self.d):
            self.velocity[i] = w * self.velocity[i] + u1[i] * (self.pos_best[i] - self.position[i]) + u2[i] * (pos_best_g[i] - self.position[i])
            
    # update the particle position
    def update_position(self):
        
        for i in range(0,self.d):
            result = self.position[i] + self.velocity[i]
            
            # check boundary conditions, reset velocity to zero if particle is on boundary
            if result < -5:
                self.position[i] = -5
                self.velocity[i] = 0
            elif result > 5:
                self.position[i] = 5
                self.velocity[i] = 0
            else:
                self.position[i] = result

In [3]:
# algorithm_class
class PSO:
    
    def __init__(self, budget):
        
        self.budget = budget
        self.d = 1
        self.num_particles = 40 
        
    def __call__(self, func):
        
        # set dimensions and budget based on problem at hand
        self.d = func.number_of_variables
        eval_budget = self.budget * self.d
        
        # Initialize global best solution and inertia weight
        best_fitness_g = np.inf         # best fitness for group
        pos_best_g = np.zeros(self.d)   # best position for group
        #w = 0.792 constant inertia weight
        
        # establish the swarm
        swarm = []
        for i in range(0,self.num_particles):
            swarm.append(Particle(self.d)) # append objects of class Particle
            swarm[i].fitness = func(swarm[i].position) # evaluate fitness for each particle
            swarm[i].pos_best = swarm[i].position.copy() # set personal_best to current position, use .copy() to avoid link
            swarm[i].best_fitness = swarm[i].fitness # set best_fitness to current particle's fitness
            
            # update global best if particle is best solution so far
            if swarm[i].fitness < best_fitness_g:
                pos_best_g = swarm[i].position.copy()
                best_fitness_g = swarm[i].fitness
                
        # evaluation loop
        while (func.evaluations < eval_budget) and not func.final_target_hit:            
         
            # update inertia weight
            w = 0.9 - 0.8 * func.evaluations/eval_budget 
            
            # iterate through particle swarm
            for k in range(0,self.num_particles):   
                swarm[k].update_velocity(pos_best_g,w) # update velocity based on global_best and inertia weight
                swarm[k].update_position() # update position with new velocity
                swarm[k].fitness = func(swarm[k].position) # evaluate the particle's fitness

                # check if new particle position is best-so-far for this particle
                if swarm[k].fitness < swarm[k].best_fitness:
                    swarm[k].pos_best = swarm[k].position.copy()
                    swarm[k].best_fitness = swarm[k].fitness

                    
                # check if new particle position is best-so-far globally
                if swarm[k].fitness < best_fitness_g:
                    pos_best_g = swarm[k].position.copy()
                    best_fitness_g = swarm[k].fitness

        # return best global solution including its fitness value
        return pos_best_g, best_fitness_g

In [ ]:
fIDs = []
for fid in range (1,25):
    fIDs.append(fid)
from IOHexperimenter import IOHexperimenter
exp = IOHexperimenter()
exp.initialize_BBOB(fIDs,[1],[2,5,10], 1)
path = "/Users/dschroeder/Documents/Master Computer Science/Master_Thesis/code_and_data/experiments/"
now = datetime.datetime.now()
experiment_ID = str(now.day) + "-" + str(now.month) + "-" + str(now.strftime('%H')) + str(now.strftime('%M')) + str(now.strftime('%S'))
experiment_name =  "PSO/experiment" + experiment_ID
exp.set_logger_location(path + experiment_name, "run")
exp([PSO(10000)])
#create a ZIP file for IOHprofiler upload
shutil.make_archive(path + experiment_name,"zip", path + experiment_name)